<a href="https://colab.research.google.com/github/danialzendehdel/Computational_Intelligence-/blob/main/Computational_intelligence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries 

In [1]:
import os
import numpy as np 
import pandas as pd 
import seaborn as sns
import gc
import tensorflow as tf 

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img, img_to_array, array_to_img
from tensorflow.keras.models import Sequential , Model 

from keras.preprocessing import image
from PIL import Image, ImageDraw
import cv2

from tensorflow.keras.callbacks import EarlyStopping , ModelCheckpoint , ReduceLROnPlateau
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D ,MaxPool2D,Input, Dropout , MaxPooling2D,Flatten,Dense,GlobalAveragePooling2D,BatchNormalization,Activation
from keras import regularizers, optimizers 
from tensorflow.keras.optimizers import Adam
from keras.regularizers import l2

from sklearn.metrics import f1_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold, train_test_split

import matplotlib.pyplot as plt
%matplotlib inline  

## tensorboard
import shutil

try:
  shutil.rmtree('logs')
except:
  pass


from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Colab configurations 
This part is specifically for running the code on Colab.


1.   Check if the google has given the premission to use GPU or not.
2.   Mount the google drive in order to import images (caustion : if you mind to use your Google Drive do not forget to download images and put them to you Drive and write the path of directory of images in code).



In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

In [ ]:
## my images directory
path = '/content/gdrive/My Drive/Kaggle'
os.chdir(path)

In [ ]:
## check the path 
pwd

# **FUNCTIONS** 

1.   `plot Bounding Box Function` : This has been only used for the output of Datagenerator for Multiple output model.
It shows images with label and bounding box just for one image at time.

2.   `result_vis` : This one plot the history of Multiple output Model
3.    `intersection_over_union` : This one measure of how much the predicted and real BBOXes have overlap 
4.    `prediction_vis` : shows the labels and bboxes
5.     `result_images` : compute the predictions base on given model and then use the `intersection_over_union` to compute IOU the with help of `prediction_vis` shows the predicted labels bboxes with real ones.  




##**Plot bounding box**

In [ ]:
def plot_bounding_box(image,gt_coords,pred_coords=None,norm=False) : 
  if norm : 
    # image *=255
    image = image.astype('uint8')
  image = Image.fromarray(image) 
  draw = ImageDraw.Draw(image)


  xmin , ymin , xmax , ymax = gt_coords
  xmin *= 224
  ymin *= 224
  xmax *= 224
  ymax *= 224

  draw.rectangle((xmin,ymin,xmax,ymax),outline='green',width=3)

  if pred_coords : 
    xminp , yminp , xmaxp , ymaxp = pred_coords
    
    xminp *= 224
    yminp *= 224
    xmaxp *= 224
    ymaxp *= 224
    draw.rectangle((xminp,yminp,xmaxp,ymaxp),outline='red',width=3)

  return image


## **result_vis**

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
def result_visual(history) : 
  
  fig = make_subplots(rows=3, cols=1,
                      subplot_titles=("Total Loss","Class Loss", "BBOX Loss"),
                      vertical_spacing=0.2)

  # subplot_titles=("Plot 1", "Plot 2", "Plot 3", "Plot 4","Plot 4")

  fig.add_trace(go.Scattergl(
                      y=history.history['loss'],
                      name='Train'),row=1, col=1)
  fig.add_trace(go.Scattergl(
                      y=history.history['val_loss'],
                      name='Valid'),row=1, col=1)

  fig.add_trace(go.Scattergl(
                      y=history.history['class_out_loss'],
                      name='Train'),row=2, col=1)
  fig.add_trace(go.Scattergl(
                      y=history.history['val_class_out_loss'],
                      name='Valid'),row=2, col=1)

  fig.add_trace(go.Scattergl(
                      y=history.history['box_out_loss'],
                      name='Train'),row=3, col=1)
  fig.add_trace(go.Scattergl(
                      y=history.history['val_box_out_loss'],
                      name='Valid'),row=3, col=1)


  fig.update_layout(height=800, width=700,
                    title_text="LOSS",
                    xaxis_title='Epoch',
                    yaxis_title='LOSS')
  # Update xaxis properties
  fig.update_xaxes(title_text="Epoch", row=1, col=1)
  fig.update_xaxes(title_text="Epoch", row=2, col=1)
  fig.update_xaxes(title_text="Epoch", showgrid=True, row=3, col=1)


  # Update yaxis properties
  fig.update_yaxes(title_text="LOSS", row=1, col=1)
  fig.update_yaxes(title_text="LOSS",  row=2, col=1)
  fig.update_yaxes(title_text="LOSS", showgrid=True, row=3, col=1)



  fig.show()

  fig2 = go.Figure()
  fig2.add_trace(go.Scattergl(
                      y=history.history['box_out_mae'],
                      name='Train'))
  fig2.add_trace(go.Scattergl(
                      y=history.history['val_box_out_mae'],
                      name='Valid'))
  fig2.update_layout(height=500, 
                    width=700,
                    title='Mean Absolute Error for age feature',
                    xaxis_title='Epoch',
                    yaxis_title='Mean Absolute Error')
  fig2.show()

  fig3 = go.Figure()
  fig3.add_trace(go.Scattergl(
                      y=history.history['class_out_accuracy'],
                      name='Train'))
  fig3.add_trace(go.Scattergl(
                      y=history.history['val_class_out_accuracy'],
                      name='Valid')
                      )
  fig3.update_layout(height=500, width=700,
                    title_text="ACCURACY",
                    xaxis_title='Epoch',
                    yaxis_title='Accuracy')



  fig3.show()

## **IoU**

In [ ]:
def intersection_over_union(pred_box, true_box):

    xmin_pred, ymin_pred, xmax_pred, ymax_pred = np.split(pred_box,4, axis = 1)
    xmin_true, ymin_true, xmax_true, ymax_true = np.split(true_box, 4, axis = 1)

    #Calculate coordinates of overlap area between boxes
    xmin_overlap = np.maximum(xmin_pred, xmin_true)
    xmax_overlap = np.minimum(xmax_pred, xmax_true)
    ymin_overlap = np.maximum(ymin_pred, ymin_true)
    ymax_overlap = np.minimum(ymax_pred, ymax_true)

    #Calculates area of true and predicted boxes
    pred_box_area = (xmax_pred - xmin_pred) * (ymax_pred - ymin_pred)
    true_box_area = (xmax_true - xmin_true) * (ymax_true - ymin_true)

    #Calculates overlap area and union area.
    overlap_area = np.maximum((xmax_overlap - xmin_overlap),0)  * np.maximum((ymax_overlap - ymin_overlap), 0)
    union_area = (pred_box_area + true_box_area) - overlap_area

    # Defines a smoothing factor to prevent division by 0
    smoothing_factor = 1e-10

    #Updates iou score
    iou = (overlap_area + smoothing_factor) / (union_area + smoothing_factor)

    return iou

## **prediction_vis**

In [ ]:
def prediction_vis(images,gt_bbox,predict_bbox,gt_label,predict_labels,iou=None): 

  # Matplotlib config
  plt.rc('image', cmap='gray')
  plt.rc('grid', linewidth=0)
  plt.rc('xtick', top=False, bottom=False, labelsize='large')
  plt.rc('ytick', left=False, right=False, labelsize='large')
  plt.rc('axes', facecolor='F8F8F8', titlesize="large", edgecolor='white')
  plt.rc('text', color='a8151a')
  plt.rc('figure', facecolor='F0F0F0')# Matplotlib fonts

  
  counter=0
  plt.figure(figsize=(10, 15))
  x = len(images)
  
  for i in range(9) : 
    counter += 1
    height , width,channel = 224,224,3
    ax = plt.subplot(3,3,i+1)
    xmin , ymin , xmax , ymax = list(gt_bbox[i])
    xminp, yminp, xmaxp, ymaxp = list(predict_bbox[i])
    image = Image.fromarray((images[i]).astype(np.uint8))
    draw = ImageDraw.Draw(image)
    draw.rectangle((int(xmin*width),int(ymin*height),int(xmax*width),int(ymax*height)),outline='green',width=3)
    draw.rectangle((int(xminp*width),int(yminp*height),int(xmaxp*width),int(ymaxp*height)),outline='red',width=3)
    plt.imshow(image)
    # if gt_label : 
    #   plt.text(int(xmin*width),
    #            int(ymin*height),
    #            target_dict2[np.argmax(gt_label[i])],
    #            backgroundcolor = 'green',
    #            horizontalalignment='right',
    #            c='black',
    #            size='large')
     
      
    plt.text(int(xmin*width),
             int(ymin*height),
             target_dict2[(gt_label[i])],
             backgroundcolor = 'green',
             horizontalalignment='right',
             c='black',
             size='large')
    plt.title('IOU : ' + str(format((iou[i][0]*100),'.2f')),size='x-large')
   
    # if predict_labels : 
    plt.text(int(xminp*width),int(yminp*height),
              target_dict2[np.argmax(predict_labels[i])],
              backgroundcolor = 'red',
              horizontalalignment='left',
              c='black',
              size='x-large')
    plt.axis("off")



##**result_images**

In [ ]:
def result_images(images,gt_bbox,gt_label,model_test=None ,model_classification = None,model_reg= None) : 
  #Makes predictions
  if model_test : 
    predicted_labels,predicted_bboxes = model_test.predict(images, batch_size=16)
  else : 
    predicted_labels = model_classification.predict(images)
    predicted_bboxes = model_reg.predict(images)

    # print(predicted_labels)
    # predicted_labels = np.argmax(predicted_labels)
  #Calculates IOU and reports true positives and false positives based on IOU threshold
  iou = intersection_over_union(predicted_bboxes, gt_bbox)
  iou_threshold = 0.5
  # print(iou)

  # print("Number of predictions where iou > threshold(%s): %s" % (iou_threshold, (iou >= iou_threshold).sum()))
  # print("Number of predictions where iou < threshold(%s): %s" % (iou_threshold, (iou < iou_threshold).sum()))

  prediction_vis(images=images,
               gt_bbox=(gt_bbox),
               predict_bbox=(predicted_bboxes),
               gt_label=gt_label,
               predict_labels=predicted_labels,
               iou=iou)

# **Dataset preparation** 
#**Caution: THERE IS NO NEED TO RUN THIS PART AT ALL AS THE RESULT OF THESE PART HAS SAVED**
The originality of this Dataset is published by **Game of Deep Learning** competition [Ship Images Dataset](https://www.kaggle.com/arpitjain007/game-of-deep-learning-ship-datasets)  , because of this images in this dataset have not specific size and the sizes are different image by image and also there are some images with lack of 3 channels we used the equal dataset which has images with 128 size with 3 channels . 

The images are imported in addition there is **.CSV** file which contains the images **filename** and **labels** which be useful just in terms of classification although in case of `Object Detection` **annotations** are needed as well so by using the benefits of `Labelimg` a new **CSV** file is created by ourself which contains _annotations_ and _Width_ and _Height_ of images in addition to previous one .The tutorial of installing and using the application is here [Labelimg](https://medium.com/deepquestai/object-detection-training-preparing-your-custom-dataset-6248679f0d1d)
Finally the format saved data is **Pascal VOC**

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"
# /content/gdrive/My Drive/Kaggle is the path where kaggle.json is present in the Google Drive

In [ ]:
cwd = os.getcwd()

# Print the current working directory
print("Current working directory: {0}".format(cwd))

# Print the type of the returned object
print("os.getcwd() returns an object of type: {0}".format(type(cwd)))

In [ ]:
 pip install kaggle --upgrade

In [ ]:
!kaggle datasets download -d canerbaloglu/ship-images-dataset

In [ ]:
# #unzipping the zip files and deleting the zip files
!unzip \*.zip  && rm *.zip

## Parse annotations 
As the annotations has been save as `.xml` format in order to make CSV file the code below has runned then the Dataset is saved .This code is taken from `Tesnorflow Object Detection TFRecord function `,you are able to see the whole in here [TFRecord](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#convert-xml-to-record)

In [ ]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET



def xml_to_csv(path):
    xml_list = []
    # for xml_file in glob.glob(path + '/*.xml'):
    for dir in os.listdir(path) :
      path_temp = (os.path.join(path,dir))
      print(path_temp)
      # for xml_file in os.listdir(path_temp) :
      for xml_file in glob.glob(path_temp + '/*.xml'):
        
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

**Save dataframe as .CSV file**

In [ ]:
path_annotation = '/content/gdrive/My Drive/Kaggle/resized_images/annotation2'

## Saved file as Dataframe
xml_df = xml_to_csv(path_annotation)


# **Data Preprocessing** 


## **Dataframe Preprocessing** 
Now as the Dataframes for both `training` and `testing` is ready,Some modification was applied in order to properly use the Dataset although some of these adjustments could be done in other places in code anyway we decided to do them in this part .
The training consists 6182 images which has been used for Training and Validation , The Testing consists 20 images which has been used just for final evaluation of each model. 

Modifications : 
1.   There was a bug with names of Labels as some of them were saved with capital alphabets so it has been fixed. 
2.   Bounding Boxes modification,so each value has divided by 128.




In [ ]:
## Read the Trainnig Dataset CSV file 
path_to_train_csv_file = '/content/gdrive/My Drive/Kaggle/resized_images/ships_kiri.csv'

train_files = pd.read_csv(path_to_train_csv_file,dtype='str')
train_files

In [ ]:
## Read the Testing Dataset CSV file 
path_to_test_csv_file = '/content/gdrive/My Drive/Kaggle/resized_images/test/ship_test.csv'

test_files = pd.read_csv(path_to_test_csv_file,dtype='str')
test_files

In [ ]:
## (1) first modification
# train bbox Normalization 
train_files['xmin'] = train_files['xmin'].apply(lambda x : int(x)/128)
train_files['ymin'] = train_files['ymin'].apply(lambda x : int(x)/128)
train_files['xmax'] = train_files['xmax'].apply(lambda x : int(x)/128)
train_files['ymax'] = train_files['ymax'].apply(lambda x : int(x)/128)

## test bbox Normalization 

test_files['xmin'] = test_files['xmin'].apply(lambda x : int(x)/128)
test_files['ymin'] = test_files['ymin'].apply(lambda x : int(x)/128)
test_files['xmax'] = test_files['xmax'].apply(lambda x : int(x)/128)
test_files['ymax'] = test_files['ymax'].apply(lambda x : int(x)/128)

In [ ]:
# count each the number images related to each category 

train_files['class'][train_files['class'] == 'Carrier'] = 'carrier'
train_files['class'][train_files['class'] == 'Military'] = 'military'
train_files['class'][train_files['class'] == 'Tanker'] = 'tanker'
train_files['class'][train_files['class'] == 'Cruise'] = 'cruise'

test_files['class'][test_files['class'] == 'criuse'] = 'cruise'

train_files.set_index(['class' , 'filename']).count(level="class")

In [ ]:
## Data visualisation as the pie chart 
import plotly.express as px
fig = px.pie(train_files, names='class', color_discrete_sequence=px.colors.sequential.RdBu)
fig.show()

In [ ]:
## Data visualisation as the bar chart for Training Data
dic = dict(train_files['class'].value_counts())
fig = go.Figure(
    data=[go.Bar(x=list(dic.keys()),y =list(train_files['class'].value_counts()))],
    layout=go.Layout(
        title=go.layout.Title(text="Quantity of each class")
    )
)

fig.show()

In [ ]:
## Data visualisation as the bar chart for Testing Data
dic = dict(test_files['class'].value_counts())
fig = go.Figure(
    data=[go.Bar(x=list(dic.keys()),y =list(test_files['class'].value_counts()))],
    layout=go.Layout(
        title=go.layout.Title(text="Quantity of each class")
    )
)

fig.show()

## **Image Preprocessing** 
In this section : 


1.   images have imported for both Training and Testing.
2.   Labels extracted from the Dataframe.
3.   Bounding Boxes has extracted from Dataframe.


The images with the help of corresponding `filename` has been read then append the array of image to a list.

In the function `ImageLoader` below

1.   The directory of folder is the input parameter containing the images for   different classes
2.   Read the image file from the folder and convert it to the right format.
3. Resize the image based on the input dimension required for the model here is 224.


In [ ]:
def imageLoader(path,img_list) : 

  data_img = []
  # img_list = list(train_files['filename'])
  for each in img_list:
    # Each image path
    each_path = os.path.join(path, each)
    # Read each imagee
    each_img = (cv2.imread(each_path))
    # OpenCv default color is BGR. Convert it to RGB
    each_img = cv2.cvtColor(each_img, cv2.COLOR_BGR2RGB)
    # Resize the images
    each_img_resized = cv2.resize(each_img, (224,224))

    # image = each_img_resized.astype('float32')
    # each_img_resized /= 255
    # Save arrays to a list
    data_img.append(each_img_resized)

  return data_img

**training section**

In [ ]:
# ground truth of bounding boxes
column = ['xmin' , 'ymin' , 'xmax' ,'ymax']
gt_bbox = train_files[column].to_numpy()
# Labels 
gt_labels = train_files['class']

## the data for further processing is saved in all_images variable 
path = '/content/gdrive/My Drive/Kaggle/all'
train_list = list(train_files['filename'])

all_images = imageLoader(path,train_list)

**Testing Section**

In [ ]:
# List of image names
column = ['xmin' , 'ymin' , 'xmax' ,'ymax']
test_list = list(test_files['filename'])
path_test = '/content/gdrive/My Drive/Kaggle/resized_images/test/image/'

test_data = imageLoader(path_test,test_list)
# Converting list to numpy array
TEST_images = np.array(test_data)
print('Shape of test data: ', TEST_images.shape)


TEST_bbox = test_files[column].to_numpy()
TEST_labels = test_files['class']

## **Convert the list of array of images to Numpy array**
Also a Dictionary base on labels has created and assign a number to each category instead of categorical labels. 

In [ ]:
## Load each images with corresponding labels and bounding boxes
def DataLoader(images , labels , bboxes) : 

  images_np = np.array(images)
  labels_np = np.array(target_val)
  bboxes_np = np.array(bboxes)

  print('type of images : ' , type(images_np),' size of images : ',images_np.shape)
  print('type of labels : ',type(labels_np),  ' size of labels : ',labels_np.shape)
  print('type of Bboxes : ',type(bboxes_np),  ' size of bboxes : ',bboxes_np.shape)

  return images_np , labels_np , bboxes_np


In [ ]:
## create a dictionary in order to assign each of 5 labels to a number 
target_dict={k: v for v, k in enumerate(np.unique(gt_labels))}
print(target_dict)

In [ ]:
target_val=  [target_dict[gt_labels[i]] for i in range(len(gt_labels))]
TEST_labels_n = [target_dict[TEST_labels[i]] for i in range(len(TEST_labels))]

In [ ]:
train_data = DataLoader(all_images,gt_labels,gt_bbox)

In [ ]:
images_all_np ,labels_all_np,bboxes_all_np = train_data

## **Visual the images with corresponding bounding box**

In [ ]:
## another dictionary for visualization of data (for np.argmax)
target_dict2 = {0 : 'cargo', 1 : 'carrier', 2 : 'cruise', 3 : 'military', 4 : 'tanker'}

In [ ]:
from PIL import Image
import cv2
from PIL import Image, ImageDraw
def visualization (img_array , label , bbox , num) : 
  counter=0
  plt.figure(figsize=(18, 28))
  x  = len(img_array)
  for i in np.random.randint(0,x,num*3) : 
    counter += 1
    height , width,channel = np.shape(img_array[i])
    ax = plt.subplot(3,num,counter)
    xmin , ymin , xmax , ymax = bbox[i]
    image = Image.fromarray((img_array[i]).astype(np.uint8))
    draw = ImageDraw.Draw(image)
    draw.rectangle((int(xmin*width),int(ymin*height),int(xmax*width),int(ymax*height)),outline='green',width=2)
    plt.imshow(image)
    plt.axis("off")
    plt.title(target_dict2[label[i]])



In [ ]:
##just for check the one single output of function 
print('shape of image :', np.shape(images_all_np[0]),'\n','type of image:', type(images_all_np[0]))

In [ ]:
images_all_np , labels_all_np,bboxes_all_np = train_data
## num is number of columns of image in subplot function , 
## number of rows is constant by 3 
num = 2

visualization(images_all_np,labels_all_np,bboxes_all_np,num)

# **Prepare Data for Models**
1.   **Split the Data for training and validation**

2.   **One Hot encoding**


80% of images used for training and 20% for Validation in shuffle mode. 

**One Hot Encoding**

In these cases, we would like to give the network more expressive power to learn a probability-like number for each possible label value. This can help in both making the problem easier for the network to model. When a one hot encoding is used for the output variable, it may offer a more nuanced set of predictions than a single label.



In [ ]:
image_train, image_test, label_train, label_test ,bbox_train , bbox_test = train_test_split(images_all_np,
                                                                                            labels_all_np,
                                                                                            bboxes_all_np,
                                                                                            test_size =0.2,
                                                                                            random_state=42)

In [ ]:
# one hot encoding 
from tensorflow.keras.utils import to_categorical

train_labels_one_hot = to_categorical(label_train)
test_labels_one_hot = to_categorical(label_test)
## check
print(train_labels_one_hot[2])
print(test_labels_one_hot[2])

#**Models** 
two models have been used :

1.   Multiple output model which has base of `MobileNetV2` 
2.   Splitted Model which has 2 models , first Xception for Classification and MobileNetV2 for regression



## **Multiple output**

### Data Generator
yield images , labels , bboxes 

In [ ]:
## Data generator 
def get_generator_train(features, labels,bbox, batch_size=16):
  while True :
    for n in range(int(len(features)/batch_size)):
        yield features[n*batch_size:(n+1)*batch_size], [labels[n*batch_size:(n+1)*batch_size],bbox[n*batch_size:(n+1)*batch_size]]
        



def get_generator_test(features, labels,bbox, batch_size=16):
  while True :
    for n in range(int(len(features)/batch_size)):
        yield features[n*batch_size:(n+1)*batch_size], [labels[n*batch_size:(n+1)*batch_size],bbox[n*batch_size:(n+1)*batch_size]]
        

In [ ]:
training_generator = get_generator_train(image_train,train_labels_one_hot,bbox_train)
testing_generator  = get_generator_test(image_test,test_labels_one_hot,bbox_test)


**So from now on we have batches of images and labels in each batch there are 16 images with size of `(16, 224, 224, 3)` and list of labels which has 2 section of `bounding boxes` and `ship class` with size of (16, 4) and (16, 5) respectively** 

In [ ]:
image, label = next(training_generator)
print(np.shape(image))
print(label[0].shape)
print(label[1].shape)

In [ ]:
## An example from training set
example , label = next(training_generator)
image = example[0]
class_id = np.argmax(label[0][0])
coords = label[1][0]
coords
image = plot_bounding_box(image,coords,norm=True)
plt.imshow(image)
plt.title(target_dict2[class_id])
plt.axis('off')
plt.show()

In [ ]:
# An example from validation set
example , label = next(testing_generator)
image = example[0]
class_id = np.argmax(label[0][0])
coords = label[1][0]
coords
image = plot_bounding_box(image,coords,norm=True)
plt.imshow(image)
plt.title(target_dict2[class_id])
plt.axis('off')
plt.show()

### Model configuration

In [ ]:
import math
EPOCHS = 15

# image_train, image_test, label_train, label_test ,bbox_train , bbox_test
# Choose a batch size
BATCH_SIZE = 16

# Get the length of the training set
length_of_training_dataset = len(image_train)

# Get the length of the validation set
length_of_validation_dataset = len(image_test)

print(length_of_training_dataset , length_of_validation_dataset)
# Get the steps per epoch (may be a few lines of code)
steps_per_epoch = math.ceil(length_of_training_dataset/BATCH_SIZE)

print(steps_per_epoch)
# get the validation steps (per epoch) (may be a few lines of code)
validation_steps = length_of_validation_dataset//BATCH_SIZE
if length_of_validation_dataset % BATCH_SIZE > 0:
    validation_steps += 1
    
print(validation_steps)

In [ ]:
# callbacks
checkpoint_path_mlp = '/content/gdrive/My Drive/Kaggle/resized_images/Multi_weights'
checkpoint_mlp = ModelCheckpoint(filepath=checkpoint_path_mlp,
monitor='val_loss', save_best_only =True, save_weights_only=True, save_freq='epoch',
                             mode='auto',
                            verbose=1)
earlystop = EarlyStopping(monitor='val_loss',patience=3,verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=1,verbose=1,min_delta=0.01)

In [ ]:
opt1 = tf.keras.optimizers.RMSprop(lr = 0.001)
opt2 = tf.keras.optimizers.SGD(learning_rate=0.01,momentum=0.9)
opt3 = tf.keras.optimizers.Adam(learning_rate=0.0002)

In [ ]:
mobilenet_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),include_top=False,weights='imagenet')
# pass the inputs into this modle object to get a feature extractor for these inputs
inputs = tf.keras.Input(shape=(224,224,3),name='image')
feature_extractor = mobilenet_model(inputs)
x2 = tf.keras.layers.GlobalAveragePooling2D()(feature_extractor)  
    
    # flatten layer
x = tf.keras.layers.Flatten()(x2)
    
    # 1024 Dense layer, with relu
x = tf.keras.layers.Dense(1024,activation='relu')(x)
    
    # 512 Dense layer, with relu
x1 = tf.keras.layers.Dense(512,activation='relu')(x)

    ## khodam ezafe kardam,###############################
y_l = tf.keras.layers.Dropout(0.3)(x)
y_l = tf.keras.layers.Dense(256,activation='relu')(x1)

y_b = tf.keras.layers.Dense(512,activation='relu')(x1)
y_b = tf.keras.layers.Dense(512,activation='relu')(y_b)
y_b = tf.keras.layers.Dropout(0.3)(y_b)
y_b = tf.keras.layers.Dense(256,activation='relu')(y_b)
y_b = tf.keras.layers.Dropout(0.3)(y_b) 
y_b = tf.keras.layers.Dense(256,activation='relu')(y_b)   

bounding_box_regression_output =tf.keras.layers.Dense(4,activation='relu',name='box_out')(y_l) 
label_output = tf.keras.layers.Dense(5,activation='softmax',name='class_out')(x2)

model = tf.keras.Model(inputs=inputs, outputs=[label_output,bounding_box_regression_output])

### Model's Summary

In [ ]:
tf.keras.utils.plot_model(model,'mult_input_output_model.png',show_shapes=True)

### Model Compling and Training

In [ ]:
model.compile(
        optimizer=opt3,
        loss = {'class_out' : 'categorical_crossentropy' ,'box_out' : 'mse'} ,
        metrics = {'class_out' : 'accuracy' ,'box_out' : 'mae'})

In [ ]:
history =  model.fit(training_generator,
                    steps_per_epoch=steps_per_epoch, 
                    validation_data = testing_generator,
                    validation_steps=validation_steps,
                    epochs=30,
                     callbacks = [checkpoint_mlp,earlystop,reduce_lr])

### result of training

In [ ]:
result_visual(history)

### Evaluation Model on test data

In [ ]:
result_images(images = TEST_images,
              gt_bbox=TEST_bbox,
              gt_label = TEST_labels_n,
              model_test=model,
              model_classification = None,
              model_reg= None)

## **Splitted Model**
This one as already said includes 2 models.

### **Classification Model** 

#### Data Generator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(horizontal_flip=True,
                               rotation_range = 45,
                               zoom_range=0.2,
                               height_shift_range = 0.5,
                               width_shift_range = 0.5)

validation_gen = ImageDataGenerator(horizontal_flip=True,
                               rotation_range = 45,
                               zoom_range=0.2,
                               height_shift_range = 0.5,
                               width_shift_range = 0.5)

train_gen.fit(image_train)
validation_gen.fit(image_test)

In [ ]:
train_gen= train_gen.flow(image_train, train_labels_one_hot, batch_size=32)
val_gen  = validation_gen.flow(image_test, test_labels_one_hot, batch_size=32)

In [ ]:
## train 
images_gen , labels_gen = train_gen[7]
plt.figure(figsize=(20, 20))
for i in range(7) : 
  ax = plt.subplot(1,7,i+1)
  plt.imshow((images_gen[i]).astype('uint8'))
  plt.title(target_dict2[(np.argmax(labels_gen[i], axis=0))])
  plt.axis("off")

In [ ]:
## test
images_val , labels_val = val_gen[1]
plt.figure(figsize=(20, 20))
for i in range(7) : 
  ax = plt.subplot(1,7,i+1)
  plt.imshow(images_val[i].astype('uint8'))
  plt.title(target_dict2[(np.argmax(labels_val[i], axis=0))])
  plt.axis("off")

#### Model configuration 

In [ ]:
from tensorflow.keras.applications.xception import Xception
def get_Class_Model() : 

  # Defining the pretrained base model
  base = Xception(include_top=False, weights='imagenet', input_shape=(224,224,3))
  x = base.output
  x = GlobalAveragePooling2D()(x)
  # Defining the head of the model where the prediction is conducted
  head = Dense(5, activation='softmax')(x)
  # Combining base and head 
  model = Model(inputs=base.input, outputs=head)
  # Compiling the model
  model.compile(optimizer=Adam(lr=0.0001), 
                loss = 'categorical_crossentropy', 
                metrics=['accuracy'])
  
  return model

In [ ]:
## callbacks
checkpoint_path = '/content/gdrive/My Drive/Kaggle/resized_images/classification_weights2'
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             monitor='val_loss', 
                             save_best_only =True, 
                             save_weights_only=True, 
                             save_freq='epoch',
                             mode='auto',
                             verbose=1)
earlystop = EarlyStopping(monitor='val_loss',min_delta=0.03,patience=3,verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.1,min_delta=0.02,patience=2,verbose=1)

#### Classification Training 

In [ ]:
classification_model = get_Class_Model()

batch_size =32
epochs = 15
model = get_Class_Model()
# Fitting the model with train and validation augmented datasets.
history = model.fit_generator(train_gen,
                              epochs = epochs,
                              validation_data = val_gen,
                              steps_per_epoch = image_train.shape[0] // batch_size,
                              callbacks= [checkpoint,reduce_lr,earlystop])

#### Model Summary

In [ ]:
tf.keras.utils.plot_model(classification_model,'classification_model_output_model.png',show_shapes=True)

#### Results

In [ ]:
## Confusion
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Predicting labels from X_test data
y_pred = model.predict(image_test)

# Converting prediction classes from one hot encoding to list
# Argmax returns the position of the largest value
y_pred_classes = np.argmax(y_pred, axis = 1)

# Convert test labels from one hot encoding to list
# y_test_classes = np.argmax(TEST_labels, axis = 1)

# Create the confusion matrix
confmx = confusion_matrix(label_test, y_pred_classes)
f, ax = plt.subplots(figsize = (8,8))
sns.heatmap(confmx, annot=True, fmt='.1f', ax = ax)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show();

In [ ]:
print(classification_report(label_test, y_pred_classes))

In [ ]:
fig2 = go.Figure()
fig2.add_trace(go.Scattergl(
                     y=history.history['accuracy'],
                     name='Train'))
fig2.add_trace(go.Scattergl(
                      y=history.history['val_accuracy'],
                      name='Valid'))
fig2.update_layout(height=500, 
                    width=700,
                    title='ACCURACY',
                    xaxis_title='Epoch',
                    yaxis_title='Accuracy')
fig2.show()


In [ ]:
fig3 = go.Figure()
fig3.add_trace(go.Scattergl(
                      y=history.history['loss'],
                      name='Train'))
fig3.add_trace(go.Scattergl(
                      y=history.history['val_loss'],
                      name='Valid')
                      )
fig3.update_layout(height=500, width=700,
                    title_text="Loss",
                    xaxis_title='Epoch',
                    yaxis_title='Loss')



fig3.show()

#### Evaluation

In [ ]:
# Predicting random 5 images
sample_pred = model_test.predict(TEST_images)
sample_classes = TEST_labels_n
# Visualizing the predictions
i = 0
plt.figure(figsize=(12,9))
for each in range(10):
    i += 1
    plt.subplot(2,5,i)
    plt.imshow(TEST_images[each])
    plt.title(target_dict2[((TEST_labels_n[each]))],color = 'green',size=15)
    plt.xlabel('PREDICTION: ' + str(target_dict2[sample_classes[each]]),size=10,color='red')
    plt.xticks([])
    plt.yticks([])
    

### **Regression Model**

#### Data Generator

In [ ]:
## Data generator 
def get_generator(features,bbox, batch_size=16):
  while True :
    for n in range(int(len(features)/batch_size)):
        yield features[n*batch_size:(n+1)*batch_size], bbox[n*batch_size:(n+1)*batch_size]

In [ ]:
training_generator = get_generator(image_train,bbox_train)
testing_generator  = get_generator(image_test,bbox_test)


In [ ]:
## An example from training set
example , bbox = next(training_generator)
image = example[0]*255
# class_id = np.argmax(label[0])
coords = bbox[0]
coords
image = plot_bounding_box(image,coords,norm=True)
plt.imshow(image)
# plt.title(target_dict2[class_id])
plt.axis('off')
plt.show()

#### Model configuration

In [ ]:
import math
EPOCHS = 15

# image_train, image_test, label_train, label_test ,bbox_train , bbox_test

BATCH_SIZE = 16

# Get the length of the training set
length_of_training_dataset = len(image_train)

# Get the length of the validation set
length_of_validation_dataset = len(image_test)

print(length_of_training_dataset , length_of_validation_dataset)
# Get the steps per epoch 
steps_per_epoch = math.ceil(length_of_training_dataset/BATCH_SIZE)

print(steps_per_epoch)
# get the validation steps (per epoch) 
validation_steps = length_of_validation_dataset//BATCH_SIZE
if length_of_validation_dataset % BATCH_SIZE > 0:
    validation_steps += 1
    
print(validation_steps)

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

def get_Reg_model() : 

  inputs = tf.keras.Input(shape=(224,224,3),name='image')
  mobilenet_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),include_top=False,weights='imagenet')
  feature_extractor = mobilenet_model(inputs)
  x = tf.keras.layers.GlobalAveragePooling2D()(feature_extractor)
  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dense(1024,activation='relu')(x)
  x = tf.keras.layers.Dense(512,activation='relu')(x)
  

  bounding_box_regression_output =tf.keras.layers.Dense(4,activation='relu')(x)
  model = tf.keras.Model(inputs=inputs, outputs=bounding_box_regression_output)

  model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01,momentum=0.9), loss="mse", metrics=["mae",'accuracy'])
  return model

In [ ]:
# callbacks
checkpoint_path_mlp = '/content/gdrive/My Drive/Kaggle/resized_images/Regression_weights2'
checkpoint_mlp = ModelCheckpoint(filepath=checkpoint_path_mlp,monitor='val_loss', 
                                 save_best_only =True, 
                                 save_weights_only=True,
                                 save_freq='epoch',
                                 mode='auto',
                                 verbose=1)
earlystop = EarlyStopping(monitor='val_loss',patience=3,verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=2,verbose=1,min_delta=0.01)

#### Regression training

In [ ]:
history_REG =  REG_model.fit(training_generator,
                    steps_per_epoch=steps_per_epoch, 
                    validation_data = testing_generator,
                    validation_steps=validation_steps,
                    epochs=30,
                     callbacks = [checkpoint_mlp,reduce_lr,earlystop])

#### Model summary

In [ ]:
# model.summary()
tf.keras.utils.plot_model(REG_model,'mult_input_output_model.png',show_shapes=True)

#### Result

In [ ]:
fig2 = go.Figure()


fig2 = go.Figure()
fig2.add_trace(go.Scattergl(
                      y=history_REG.history['loss'],
                      name='Train'))
fig2.add_trace(go.Scattergl(
                      y=history_REG.history['val_loss'],
                      name='Valid'))
fig2.update_layout(height=500, 
                    width=700,
                    title='LOSS',
                    xaxis_title='Epoch',
                    yaxis_title='Loss')
fig2.show()

fig3 = go.Figure()
fig3.add_trace(go.Scattergl(
                      y=history_REG.history['mae'],
                      name='Train'))
fig3.add_trace(go.Scattergl(
                      y=history_REG.history['val_mae'],
                      name='Valid')
                      )
fig3.update_layout(height=500, width=700,
                    title_text="Mean Absolute Error for age feature",
                    xaxis_title='Epoch',
                    yaxis_title='Mean Absolute Error')



fig3.show()

#### Evaluation

In [ ]:
prediction_bbox = REG_model.predict(TEST_images)

In [ ]:
plot_bounding_box(TEST_images[4],list(TEST_bbox[4]),list(prediction_bbox[4]))

## **Wrap all together** 

In [ ]:
## regressiom model
path_reg = '/content/gdrive/My Drive/Kaggle/resized_images/Regression_weights2'
model_reg2 = get_Reg_model()
model_reg2.load_weights(path_reg)

## classification model
model_classification = get_Class_Model()
model_classification.load_weights('/content/gdrive/My Drive/Kaggle/resized_images/classification_weights2')

In [ ]:
result_images(images=TEST_images,
              gt_bbox = TEST_bbox,
              gt_label = TEST_labels_n,
              model_classification=model_classification,
              model_test = None,
              model_reg = model_reg2)